# 6.2 Стекинг

In [8]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [29]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [30]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [31]:
df.shape

(10000, 55)

In [32]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [28]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.5)

In [6]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [19]:
cover_X_test

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
9460,3268,176,12,323,38,4447,226,246,150,2418,...,0,0,1,0,0,0,0,0,0,0
7852,3430,225,10,824,162,3535,206,250,180,306,...,0,0,0,0,0,0,0,1,0,0
1966,2890,315,7,342,91,1595,201,234,172,1869,...,0,0,0,0,0,0,0,0,0,0
1395,2974,6,10,216,49,1047,208,222,150,2192,...,0,0,0,0,0,0,0,0,0,0
7285,3150,338,17,342,78,3120,182,214,171,2503,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7129,2974,99,22,277,-33,488,250,204,71,966,...,0,0,0,0,0,0,0,0,0,0
1299,2890,68,17,331,-37,540,235,204,95,1884,...,0,0,0,0,0,0,0,0,0,0
1417,3215,28,23,175,35,5523,204,183,105,1366,...,0,0,0,0,0,0,0,0,0,0
1418,3221,219,3,256,25,6006,216,243,164,1622,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
count,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,...,2837.000000,2837.000000,2837.000000,2837.000000,2837.000000,2837.0,2837.000000,2837.000000,2837.000000,2837.000000
mean,3017.875220,155.593232,12.850899,277.732464,43.927388,2583.450828,213.277053,224.611562,143.671836,2108.287980,...,0.117378,0.073669,0.001410,0.000352,0.000352,0.0,0.020092,0.015509,0.009870,1.515333
std,192.408465,112.875642,6.817279,217.503394,57.419242,1600.238163,24.573931,17.850203,35.096258,1370.782364,...,0.321926,0.261278,0.037529,0.018775,0.018775,0.0,0.140339,0.123589,0.098872,0.499853
min,2169.000000,0.000000,0.000000,0.000000,-123.000000,30.000000,86.000000,115.000000,0.000000,30.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
25%,2896.000000,58.000000,8.000000,108.000000,6.000000,1271.000000,201.000000,215.000000,123.000000,1154.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
50%,3027.000000,123.000000,12.000000,234.000000,26.000000,2263.000000,218.000000,227.000000,143.000000,1848.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000
75%,3163.000000,266.000000,17.000000,391.000000,63.000000,3666.000000,231.000000,237.000000,168.000000,2672.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000
max,3675.000000,359.000000,45.000000,1343.000000,554.000000,6890.000000,254.000000,254.000000,248.000000,6853.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,2.000000


In [7]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

NameError: name 'StandardScaler' is not defined

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [43]:
train_y

734     2
9574    2
1409    1
102     2
9684    2
       ..
8210    1
6481    2
493     2
1355    1
769     2
Name: 54, Length: 1418, dtype: int64

In [50]:
cv = KFold(n_splits=10, shuffle=True)

splits = cv.split(train_X)
for each in splits:
    print(len(each))

2
2
2
2
2
2
2
2
2
2


In [51]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    # we check how many variables we have in the target column
    n_classes = len(np.unique(y_train))
    # create an array where #rows - the # of rows in the original y_train and the #columns - the # of the classes in the target feature
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        # for each iteration we get different indices for the train and test data
        X_fold_train, y_fold_train = X_train[train_fold_index], y_train[train_fold_index]
        X_fold_predict = X_train[predict_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [52]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test += folded_clf.predict_proba(X_test)

    X_meta_test /= cv.n_splits
    
    return X_meta_train, X_meta_test

In [54]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [32]:
np.random.seed(42)

In [12]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7914023960535589

In [13]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.65s/it]


In [14]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [15]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.8083157152924595

In [39]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', multi_class='multinomial', max_iter=2000),
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=200)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')
# clf.fit(stacked_features_train, cover_y_train)
# new_features = clf.predict(stacked_features_test)

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=cover_y_train,  X_test=stacked_features_test, y_test=cover_y_test)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.10s/it]


0.806864

In [33]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [34]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [56]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', multi_class='multinomial', max_iter=2000),
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=200)
], X_train, X_test, y_train, cv)

# total_features_train = np.hstack([X_train, stacked_features_train])
# total_features_test = np.hstack([X_test, stacked_features_test])

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')
# clf.fit(stacked_features_train, cover_y_train)
# new_features = clf.predict(stacked_features_test)

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:38<00:00, 39.63s/it]
c:\users\voytik\anaconda3\envs\ds\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.982421

In [59]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=200, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.75s/it]
c:\users\voytik\anaconda3\envs\ds\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.987157

In [60]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.88s/it]
c:\users\voytik\anaconda3\envs\ds\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9878

In [62]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000),
    KNeighborsClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
    AdaBoostClassifier(random_state=42)
], X_train, X_test, y_train, cv)

# total_features_train = np.hstack([X_train, stacked_features_train])
# total_features_test = np.hstack([X_test, stacked_features_test])

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')
# clf.fit(stacked_features_train, cover_y_train)
# new_features = clf.predict(stacked_features_test)

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:47<00:00, 11.80s/it]


0.989752

In [68]:
# use only for the stratified example
# def compute_meta_feature(clf, X_train, X_test, y_train, cv):
#     """
#     Computes meta-features using the classifier.
    
#     :arg clf: scikit-learn classifier
#     :args X_train, y_train: training set
#     :arg X_test: testing set
#     :arg cv: cross-validation folding
#     """
#     # we check how many variables we have in the target column
#     n_classes = len(np.unique(y_train))
#     # create an array where #rows - the # of rows in the original y_train and the #columns - the # of the classes in the target feature
#     X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

#     splits = cv.split(X_train, y_train)
#     for train_fold_index, predict_fold_index in splits:
#         # for each iteration we get different indices for the train and test data
#         X_fold_train, y_fold_train = X_train[train_fold_index], y_train[train_fold_index]
#         X_fold_predict = X_train[predict_fold_index]
        
#         folded_clf = clone(clf)
#         folded_clf.fit(X_fold_train, y_fold_train)
        
#         X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
#     meta_clf = clone(clf)
#     meta_clf.fit(X_train, y_train)
    
#     X_meta_test = meta_clf.predict_proba(X_test)
    
#     return X_meta_train, X_meta_test

In [69]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.52s/it]


0.981637

In [70]:
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.69s/it]


0.983371

In [93]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = RandomForestClassifier(n_jobs=-1, random_state=42)

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.54s/it]


0.982334

In [73]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = GradientBoostingClassifier()

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.38s/it]


0.986659

In [74]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)

np.random.seed(42)
clf = ExtraTreesClassifier(n_estimators=100, n_jobs=-1)

def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train,  X_test=stacked_features_test, y_test=y_test)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.28s/it]


0.982528

In [92]:
rf = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, n_jobs=-1)
rf.fit(X_train, y_train)
rf_y_test_pred = rf.predict_proba(X_test)

et = ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
et.fit(X_train, y_train)
et_y_test_pred = et.predict_proba(X_test)

lr = LogisticRegression(n_jobs=-1)
lr.fit(X_train, y_train)
lr_y_test_pred = lr.predict_proba(X_test)

y_test_pred = (rf_y_test_pred + et_y_test_pred + lr_y_test_pred).argmax(axis=1)
np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

0.976259